# SO-101 Auto-Calibration — Choreographed Sequence

Automatically calibrates each joint by slowly driving it into its hard stops and detecting the load spike.
No manual joint movement required — just position the arm vertically and run the cells.

### How it works
1. **Only the joint being calibrated has torque enabled** — no brute-force holding of all joints
2. Joints are calibrated in a choreographed order with **repositioning moves** between steps to avoid collisions
3. After shoulder_lift, the **wrist is moved fully upward** to give the elbow clearance to sweep
4. Before shoulder_pan, **all joints are centred** so the arm is compact and sweeps clear
5. `wrist_roll` is continuous (no hard stops) — uses `set_half_turn_homings()` with full 0–4095 range

### Calibration sequence
| Phase | Joint | Notes |
|-------|-------|-------|
| 1 | wrist_roll | Continuous joint, half-turn homing |
| 2 | gripper | Tip of arm, zero collision risk |
| 3 | wrist_flex | Small joint, tip only |
| 4 | shoulder_lift | Arm is vertical, shoulder tilts safely → return to middle |
| 5 | elbow_flex | Wrist moved fully upward for clearance, shoulder at middle |
| 6 | *(reposition)* | Centre ALL joints (gripper, wrist, shoulder, elbow) |
| 7 | shoulder_pan | Everything centred, arm compact, base rotation sweeps clear |

### Safety
- **Before starting**: position the arm **straight up (vertical)** with the gripper pointing at the ceiling
- Torque is capped at ~30-80% during calibration, with 30s timeout per direction
- `Ctrl+C` or kernel interrupt will safely disable torque and restore settings

In [ ]:
import time
from pathlib import Path

from lerobot.motors.feetech.feetech import FeetechMotorsBus
from lerobot.motors.motors_bus import Motor, MotorCalibration, MotorNormMode

# ── Per-joint calibration defaults (tuned for STS3215 30kg/cm) ────────
JOINT_DEFAULTS = {
    "gripper":       {"max_torque": 500, "load_threshold": 350, "grace_s": 0.5},
    "wrist_flex":    {"max_torque": 500, "load_threshold": 350, "grace_s": 0.5},
    "elbow_flex":    {"max_torque": 800, "load_threshold": 400, "grace_s": 1.5},
    "shoulder_lift": {"max_torque": 800, "load_threshold": 450, "grace_s": 2.0},
    "shoulder_pan":  {"max_torque": 800, "load_threshold": 400, "grace_s": 1.5},
}

MOVE_TORQUE = 600
POS_MIN = 0
POS_MAX = 4095


def _save_motor_settings(bus: FeetechMotorsBus, motor: str) -> dict:
    return {
        "Max_Torque_Limit": bus.read("Max_Torque_Limit", motor, normalize=False),
        "Torque_Limit": bus.read("Torque_Limit", motor, normalize=False),
        "Goal_Velocity": bus.read("Goal_Velocity", motor, normalize=False),
        "Acceleration": bus.read("Acceleration", motor, normalize=False),
        "P_Coefficient": bus.read("P_Coefficient", motor, normalize=False),
        "I_Coefficient": bus.read("I_Coefficient", motor, normalize=False),
    }


def _safe_disable_torque(bus: FeetechMotorsBus, motor: str | None = None) -> None:
    for attempt in range(3):
        try:
            if motor:
                bus.disable_torque(motor)
            else:
                bus.disable_torque()
            return
        except (RuntimeError, ConnectionError):
            time.sleep(0.1)
    try:
        if motor:
            bus.write("Torque_Enable", motor, 0, normalize=False)
        else:
            for m in bus.motors:
                try:
                    bus.write("Torque_Enable", m, 0, normalize=False)
                except (RuntimeError, ConnectionError):
                    pass
    except (RuntimeError, ConnectionError):
        pass


def _hold_at_current(bus: FeetechMotorsBus, motor: str, torque: int = MOVE_TORQUE, verbose: bool = True) -> None:
    """Enable torque on a motor and hold it at its current position."""
    pos = bus.read("Present_Position", motor, normalize=False)
    bus.write("Goal_Position", motor, pos, normalize=False)
    bus.enable_torque(motor)
    bus.write("Torque_Limit", motor, torque, normalize=False)
    if verbose:
        print(f"    Holding {motor} at {pos}")


def _find_hard_stop(
    bus: FeetechMotorsBus,
    motor: str,
    direction: int,
    baseline_load: float,
    load_threshold: int,
    step_size: int = 10,
    load_count: int = 4,
    stall_count: int = 15,
    grace_s: float = 0.5,
    timeout_s: float = 30.0,
    verbose: bool = True,
) -> int:
    start_pos = bus.read("Present_Position", motor, normalize=False)
    goal = start_pos
    prev_pos = start_pos
    stalls = 0
    consecutive_loads = 0
    t0 = time.monotonic()
    stop_reason = "timeout"

    while time.monotonic() - t0 < timeout_s:
        goal += direction * step_size

        if goal < POS_MIN:
            goal = POS_MIN
            stop_reason = f"encoder boundary ({POS_MIN})"
            bus.write("Goal_Position", motor, goal, normalize=False)
            time.sleep(0.2)
            break
        if goal > POS_MAX:
            goal = POS_MAX
            stop_reason = f"encoder boundary ({POS_MAX})"
            bus.write("Goal_Position", motor, goal, normalize=False)
            time.sleep(0.2)
            break

        bus.write("Goal_Position", motor, goal, normalize=False)
        time.sleep(0.05)

        try:
            cur_pos = bus.read("Present_Position", motor, normalize=False)
            cur_load = bus.read("Present_Load", motor, normalize=False)
        except (RuntimeError, ConnectionError) as e:
            stop_reason = f"servo error: {e}"
            break

        elapsed = time.monotonic() - t0
        opposing_load = (-direction) * (cur_load - baseline_load)

        if opposing_load > load_threshold:
            consecutive_loads += 1
            if consecutive_loads >= load_count:
                stop_reason = (
                    f"load={cur_load} sustained {consecutive_loads}x "
                    f"(baseline={baseline_load:.0f}, opposing={opposing_load:.0f})"
                )
                break
        else:
            consecutive_loads = 0

        if elapsed > grace_s:
            if abs(cur_pos - prev_pos) < 2:
                stalls += 1
                if stalls >= stall_count:
                    stop_reason = f"stall at {cur_pos}"
                    break
            else:
                stalls = 0

        prev_pos = cur_pos

    try:
        final_pos = bus.read("Present_Position", motor, normalize=False)
    except (RuntimeError, ConnectionError):
        final_pos = prev_pos

    if verbose:
        dir_label = "+" if direction > 0 else "-"
        elapsed = time.monotonic() - t0
        print(f"    [{dir_label}] stopped at {final_pos} after {elapsed:.1f}s ({stop_reason})")
    return final_pos


def _calibrate_joint(
    bus: FeetechMotorsBus,
    motor: str,
    step_size: int = 10,
    goal_velocity: int = 100,
    acceleration: int = 10,
    stall_count: int = 15,
    load_count: int = 4,
    timeout_s: float = 30.0,
    verbose: bool = True,
) -> tuple[MotorCalibration, int, int]:
    """Calibrate one joint: set_half_turn_homings from current position, then sweep both directions."""
    defaults = JOINT_DEFAULTS.get(motor, {"max_torque": 400, "load_threshold": 350, "grace_s": 1.0})
    max_torque = defaults["max_torque"]
    load_threshold = defaults["load_threshold"]
    grace_s = defaults["grace_s"]

    if verbose:
        print(f"{'='*50}")
        print(f"  {motor} (torque={max_torque}, threshold={load_threshold}, grace={grace_s}s)")
        print(f"{'='*50}")

    _safe_disable_torque(bus, motor)
    ht_offsets = bus.set_half_turn_homings([motor])
    ht_offset = ht_offsets[motor]

    start_pos = bus.read("Present_Position", motor, normalize=False)
    loads = []
    for _ in range(10):
        loads.append(bus.read("Present_Load", motor, normalize=False))
        time.sleep(0.02)
    baseline_load = sum(loads) / len(loads)
    if verbose:
        print(f"    start_pos={start_pos} (ht_offset={ht_offset}), baseline_load={baseline_load:.1f}")

    bus.write("Max_Torque_Limit", motor, max_torque, normalize=False)
    bus.write("Acceleration", motor, acceleration, normalize=False)
    bus.write("Goal_Velocity", motor, goal_velocity, normalize=False)
    bus.write("Goal_Position", motor, start_pos, normalize=False)
    bus.enable_torque(motor)
    bus.write("Torque_Limit", motor, max_torque, normalize=False)
    time.sleep(0.3)

    # Give shoulder_lift a kick to break static friction after gravity sag
    if motor == "shoulder_lift":
        kick = start_pos + 50
        kick = max(POS_MIN, min(POS_MAX, kick))
        bus.write("Goal_Position", motor, kick, normalize=False)
        time.sleep(0.5)
        bus.write("Goal_Position", motor, start_pos, normalize=False)
        time.sleep(0.3)
        if verbose:
            actual = bus.read("Present_Position", motor, normalize=False)
            print(f"    after kick: pos={actual}")

    if verbose:
        print(f"    Searching negative direction...")
    limit_neg = _find_hard_stop(
        bus, motor, -1, baseline_load, load_threshold,
        step_size=step_size, load_count=load_count, stall_count=stall_count,
        grace_s=grace_s, timeout_s=timeout_s, verbose=verbose,
    )

    if verbose:
        print(f"    Searching positive direction...")
    limit_pos = _find_hard_stop(
        bus, motor, +1, baseline_load, load_threshold,
        step_size=step_size, load_count=load_count, stall_count=stall_count,
        grace_s=grace_s, timeout_s=timeout_s, verbose=verbose,
    )

    range_min = min(limit_neg, limit_pos)
    range_max = max(limit_neg, limit_pos)
    center = (range_min + range_max) // 2
    final_homing_offset = ht_offset + (center - 2047)

    if verbose:
        total_range = range_max - range_min
        print(f"    range=[{range_min}, {range_max}] ({total_range} ticks)")
        print(f"    center={center}, final_offset={final_homing_offset}")

    cal = MotorCalibration(
        id=bus.motors[motor].id,
        drive_mode=0,
        homing_offset=final_homing_offset,
        range_min=range_min - (center - 2047),
        range_max=range_max - (center - 2047),
    )

    return cal, range_min, range_max


def _move_joint_to(
    bus: FeetechMotorsBus,
    motor: str,
    position: int,
    torque: int = MOVE_TORQUE,
    velocity: int = 200,
    acceleration: int = 10,
    settle_s: float = 1.5,
    verbose: bool = True,
) -> None:
    position = max(POS_MIN, min(POS_MAX, position))
    if verbose:
        print(f"    Moving {motor} to {position}...")
    bus.write("Torque_Limit", motor, torque, normalize=False)
    bus.write("Goal_Velocity", motor, velocity, normalize=False)
    bus.write("Acceleration", motor, acceleration, normalize=False)
    bus.write("Goal_Position", motor, position, normalize=False)
    time.sleep(settle_s)
    actual = bus.read("Present_Position", motor, normalize=False)
    if verbose:
        print(f"    {motor} at {actual} (target {position})")


def auto_calibrate(
    bus: FeetechMotorsBus,
    continuous_joints: list[str] | None = None,
    step_size: int = 10,
    goal_velocity: int = 100,
    acceleration: int = 10,
    stall_count: int = 15,
    load_count: int = 4,
    timeout_per_direction_s: float = 30.0,
    save_path: Path | str | None = None,
    verbose: bool = True,
) -> dict[str, MotorCalibration]:
    if continuous_joints is None:
        continuous_joints = ["wrist_roll"]

    calibration: dict[str, MotorCalibration] = {}
    joint_centers: dict[str, int] = {}

    all_saved = {}
    for motor in bus.motors:
        all_saved[motor] = _save_motor_settings(bus, motor)

    cal_kw = dict(
        step_size=step_size, goal_velocity=goal_velocity,
        acceleration=acceleration, stall_count=stall_count,
        load_count=load_count, timeout_s=timeout_per_direction_s,
        verbose=verbose,
    )

    try:
        # ── Phase 1: wrist_roll (continuous) ──────────────────────────
        for motor in continuous_joints:
            if motor not in bus.motors:
                continue
            if verbose:
                print(f"\n  Phase 1: {motor} (continuous)")
            _safe_disable_torque(bus, motor)
            bus.reset_calibration([motor])
            homing_offsets = bus.set_half_turn_homings([motor])
            calibration[motor] = MotorCalibration(
                id=bus.motors[motor].id, drive_mode=0,
                homing_offset=homing_offsets[motor], range_min=0, range_max=4095,
            )
            if verbose:
                print(f"    offset={homing_offsets[motor]}, range=0..4095\n")

        # ── Phase 2: gripper ──────────────────────────────────────────
        if "gripper" in bus.motors:
            cal, rmin, rmax = _calibrate_joint(bus, "gripper", **cal_kw)
            calibration["gripper"] = cal
            joint_centers["gripper"] = (rmin + rmax) // 2
            _move_joint_to(bus, "gripper", joint_centers["gripper"], verbose=verbose)

        # ── Phase 3: wrist_flex ───────────────────────────────────────
        wrist_flex_limit_neg = None
        if "wrist_flex" in bus.motors:
            cal, rmin, rmax = _calibrate_joint(bus, "wrist_flex", **cal_kw)
            calibration["wrist_flex"] = cal
            joint_centers["wrist_flex"] = (rmin + rmax) // 2
            wrist_flex_limit_neg = rmin
            _move_joint_to(bus, "wrist_flex", joint_centers["wrist_flex"], verbose=verbose)

        # ── Phase 4: shoulder_lift ────────────────────────────────────
        if "shoulder_lift" in bus.motors:
            cal, rmin, rmax = _calibrate_joint(bus, "shoulder_lift", **cal_kw)
            calibration["shoulder_lift"] = cal
            shoulder_center = (rmin + rmax) // 2
            joint_centers["shoulder_lift"] = shoulder_center
            if verbose:
                print(f"    Returning shoulder to center...")
            _move_joint_to(bus, "shoulder_lift", shoulder_center, torque=800, settle_s=2.5, verbose=verbose)
            actual = bus.read("Present_Position", "shoulder_lift", normalize=False)
            if abs(actual - shoulder_center) > 100:
                if verbose:
                    print(f"    Retrying shoulder centre...")
                _move_joint_to(bus, "shoulder_lift", shoulder_center, torque=800, settle_s=3.0, verbose=verbose)

        # ── Phase 5: reposition wrist + elbow_flex ────────────────────
        if "elbow_flex" in bus.motors:
            if verbose:
                print(f"\n  Repositioning for elbow calibration:")

            # Move wrist to its negative limit to clear elbow sweep
            if wrist_flex_limit_neg is not None:
                if verbose:
                    print(f"  Moving wrist_flex to neg limit ({wrist_flex_limit_neg}) for clearance:")
                _move_joint_to(bus, "wrist_flex", wrist_flex_limit_neg, verbose=verbose)

            # Hold shoulder_pan at current position so it doesn't dangle
            if "shoulder_pan" in bus.motors:
                _hold_at_current(bus, "shoulder_pan", verbose=verbose)

            # Also hold shoulder_lift at center so it doesn't sag
            if "shoulder_lift" in joint_centers:
                _move_joint_to(bus, "shoulder_lift", joint_centers["shoulder_lift"],
                               torque=800, settle_s=1.0, verbose=verbose)

            cal, rmin, rmax = _calibrate_joint(bus, "elbow_flex", **cal_kw)
            calibration["elbow_flex"] = cal
            joint_centers["elbow_flex"] = (rmin + rmax) // 2
            _move_joint_to(bus, "elbow_flex", joint_centers["elbow_flex"], verbose=verbose)

        # ── Centre ALL joints before shoulder_pan ─────────────────────
        if verbose:
            print(f"\n  Centring all joints before shoulder_pan:")
        for jname, jcenter in joint_centers.items():
            t = 800 if "shoulder" in jname else MOVE_TORQUE
            _move_joint_to(bus, jname, jcenter, torque=t, settle_s=1.0, verbose=verbose)

        # ── Phase 6: shoulder_pan ─────────────────────────────────────
        if "shoulder_pan" in bus.motors:
            cal, rmin, rmax = _calibrate_joint(bus, "shoulder_pan", **cal_kw)
            calibration["shoulder_pan"] = cal
            joint_centers["shoulder_pan"] = (rmin + rmax) // 2
            _move_joint_to(bus, "shoulder_pan", joint_centers["shoulder_pan"], verbose=verbose)

        # ── Write calibration ─────────────────────────────────────────
        _safe_disable_torque(bus)
        bus.write_calibration(calibration)
        if verbose:
            print(f"\n{'='*50}")
            print("  Calibration written to motors")
            print(f"{'='*50}")
            for name, c in calibration.items():
                rng = c.range_max - c.range_min
                print(f"    {name}: offset={c.homing_offset}, range=[{c.range_min}, {c.range_max}] ({rng} ticks)")

    except KeyboardInterrupt:
        print(f"\n  Interrupted! Disabling all torque...")
        _safe_disable_torque(bus)
        raise
    finally:
        _safe_disable_torque(bus)
        for motor, saved in all_saved.items():
            for reg, val in saved.items():
                try:
                    bus.write(reg, motor, val, normalize=False)
                except (RuntimeError, ConnectionError):
                    pass

    if save_path is not None:
        import json
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        cal_dict = {
            name: {"id": c.id, "drive_mode": c.drive_mode,
                   "homing_offset": c.homing_offset,
                   "range_min": c.range_min, "range_max": c.range_max}
            for name, c in calibration.items()
        }
        save_path.write_text(json.dumps(cal_dict, indent=2))
        if verbose:
            print(f"  Saved to {save_path}")

    return calibration

In [48]:
# ── Port detection (reused from teleop.ipynb) ─────────────────────────
import glob
import scservo_sdk as scs

VOLTAGE_ADDR = 62  # Present_Voltage, 1 byte, unit = 0.1V

ports = sorted(glob.glob("/dev/ttyACM*") + glob.glob("/dev/ttyUSB*"))
if not ports:
    print("No serial ports found! Are the arms plugged in?")

leader_port = None
follower_port = None

for port_path in ports:
    port = scs.PortHandler(port_path)
    if not port.openPort():
        continue
    port.setBaudRate(1000000)
    ph = scs.PacketHandler(0)
    data, comm, _ = ph.readTxRx(port, 1, VOLTAGE_ADDR, 1)
    port.closePort()

    if comm != 0:
        continue

    volts = data[0] / 10.0
    if volts > 8:
        label = "FOLLOWER (~12V)"
        follower_port = port_path
    else:
        label = "LEADER  (~5V)"
        leader_port = port_path
    print(f"  {port_path}: {volts:.1f}V  ->  {label}")

if leader_port and follower_port:
    print("Ports detected OK")
elif leader_port:
    print("Only leader detected")
elif follower_port:
    print("Only follower detected")
else:
    print("ERROR: No arms detected")

  /dev/ttyACM1: 5.2V  ->  LEADER  (~5V)
  /dev/ttyACM2: 12.4V  ->  FOLLOWER (~12V)
Ports detected OK


## Follower Calibration

**Before running**: position the follower arm **straight up (vertical)** with the gripper pointing at the ceiling.

In [49]:
assert follower_port, "No follower port detected — run port detection cell first"

norm_mode_body = MotorNormMode.RANGE_M100_100

follower_bus = FeetechMotorsBus(
    port=follower_port,
    motors={
        "shoulder_pan":  Motor(1, "sts3215", norm_mode_body),
        "shoulder_lift": Motor(2, "sts3215", norm_mode_body),
        "elbow_flex":    Motor(3, "sts3215", norm_mode_body),
        "wrist_flex":    Motor(4, "sts3215", norm_mode_body),
        "wrist_roll":    Motor(5, "sts3215", norm_mode_body),
        "gripper":       Motor(6, "sts3215", MotorNormMode.RANGE_0_100),
    },
)
follower_bus.connect()
print(f"Connected to follower on {follower_port}")

follower_cal = auto_calibrate(follower_bus)

follower_bus.disconnect()
print("\nFollower disconnected.")

Connected to follower on /dev/ttyACM2

  Phase 1: wrist_roll (continuous)
    offset=1902, range=0..4095

  gripper (torque=500, threshold=350, grace=0.5s)
    start_pos=2047 (ht_offset=1537), baseline_load=0.0
    Searching negative direction...
    [-] stopped at 1779 after 2.1s (load=496 sustained 4x (baseline=0, opposing=496))
    Searching positive direction...
    [+] stopped at 3447 after 9.3s (load=-436 sustained 4x (baseline=0, opposing=436))
    range=[1779, 3447] (1668 ticks)
    center=2613, final_offset=2103
    Moving gripper to 2613...
    gripper at 2792 (target 2613)
  wrist_flex (torque=500, threshold=350, grace=0.5s)
    start_pos=2047 (ht_offset=-1593), baseline_load=0.0
    Searching negative direction...
    [-] stopped at 764 after 7.3s (load=428 sustained 4x (baseline=0, opposing=428))
    Searching positive direction...
    [+] stopped at 3250 after 13.5s (load=-408 sustained 4x (baseline=0, opposing=408))
    range=[764, 3250] (2486 ticks)
    center=2007, fin

ValueError: Magnitude 2103 exceeds 2047 (max for sign_bit_index=11)

## Leader Calibration

**Before running**: position the leader arm **straight up (vertical)** with the gripper pointing at the ceiling.

In [ ]:
assert leader_port, "No leader port detected — run port detection cell first"

norm_mode_body = MotorNormMode.RANGE_M100_100

leader_bus = FeetechMotorsBus(
    port=leader_port,
    motors={
        "shoulder_pan":  Motor(1, "sts3215", norm_mode_body),
        "shoulder_lift": Motor(2, "sts3215", norm_mode_body),
        "elbow_flex":    Motor(3, "sts3215", norm_mode_body),
        "wrist_flex":    Motor(4, "sts3215", norm_mode_body),
        "wrist_roll":    Motor(5, "sts3215", norm_mode_body),
        "gripper":       Motor(6, "sts3215", MotorNormMode.RANGE_0_100),
    },
)
leader_bus.connect()
print(f"Connected to leader on {leader_port}")

leader_cal = auto_calibrate(leader_bus)

leader_bus.disconnect()
print("\nLeader disconnected.")

Connected to leader on /dev/ttyACM1
  Phase 1: wrist_roll (continuous — half-turn homing)
    homing_offset=-1269, range=0..4095

  Phase 2: gripper
  gripper (torque=500, threshold=350, grace=0.5s)
    start_pos=917, baseline_load=0.0
    Searching negative direction (expecting +load)...
    [-] stopped at 908 after 1.1s (load=404 sustained 4x (baseline=0, thresh=350, opposing=404))
    Searching positive direction (expecting -load)...
    [+] stopped at 1481 after 6.8s (servo error: Failed to read 'Present_Position' on id_=6 after 1 tries. [RxPacketError] Input voltage error!)
    range=[908, 1481] (573 ticks)
    center=1194, homing_offset=-853
    Moving gripper to 1194...
    gripper at 1264 (target was 1194)

  Phase 3: wrist_flex
  wrist_flex (torque=500, threshold=350, grace=0.5s)
    start_pos=2409, baseline_load=0.0
    Searching negative direction (expecting +load)...

  Interrupted! Disabling all torque...


KeyboardInterrupt: 

In [ ]:
# ── Verification: read back calibration and print summary ─────────────
print("=" * 60)
print("  Calibration Summary")
print("=" * 60)

for label, cal in [("Follower", follower_cal), ("Leader", leader_cal)]:
    print(f"\n  {label}:")
    print(f"    {'Joint':<15} {'ID':>3} {'Offset':>8} {'Min':>6} {'Max':>6} {'Range':>6}")
    print(f"    {'-'*15} {'-'*3} {'-'*8} {'-'*6} {'-'*6} {'-'*6}")
    for name, c in cal.items():
        rng = c.range_max - c.range_min
        print(f"    {name:<15} {c.id:>3} {c.homing_offset:>8} {c.range_min:>6} {c.range_max:>6} {rng:>6}")